In [34]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import openpyxl
import re
from datetime import datetime
from pathlib import Path
import os
from io import BytesIO
from azure.storage.blob import BlobServiceClient

# from planification.pool import *
from planification.pool.base_pool_proj import read_base_pool_proj
from planification.pool.comp_arrivals import read_pool_component_arrivals
from planification.pool.comp_changeouts import read_cc
from planification.pool.generate_projection import (
    allocate_components,
    generate_pool_projection,
)

import numpy as np
from datetime import datetime, timedelta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import hashlib

print(hashlib.sha256("komhive".encode()).hexdigest())

7f75db3f36562e7721f0e5137b045f2a3a39b76dbe19af40d2f67a4dd95aedcc


In [25]:
pool_proj_df = read_base_pool_proj()

[autoreload of planification.pool.base_pool_proj failed: Traceback (most recent call last):
  File "C:\Users\andmn\DataspellProjects\koapp\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "C:\Users\andmn\DataspellProjects\koapp\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\andmn\DataspellProjects\koapp\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "C:\Users\andmn\DataspellProjects\koapp\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "C:\Users\andmn\DataspellProjects\koapp\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment only supported for mutable types or

In [26]:
pool_proj_df

,pool_slot,changeout_week,equipo,component_serial,changeout_date,arrival_week,pool_changeout_type,arrival_date,component_code
0,1,2023-W27,856,#EG08090003,2023-07-03,2023-W40,I,2023-10-02,bp
1,2,2023-W27,881,#EN12040115,2023-07-03,2023-W34,P,2023-08-21,bp
2,3,2023-W27,875,#EN11110073,2023-07-03,2023-W38,I,2023-09-18,bp
3,4,2023-W28,883,#EN12090007,2023-07-10,2023-W38,I,2023-09-18,bp
4,6,2023-W29,883,#EE06050200-1,2023-07-17,2023-W42,I,2023-10-16,bp
...,...,...,...,...,...,...,...,...,...
14,7,2024-W14,860,#66303280,2024-04-01,2024-W30,P,2024-07-22,mp
15,4,2024-W20,861,#66303007,2024-05-13,2024-W36,P,2024-09-02,mp
16,2,2024-W21,848,#66303005,2024-05-20,2024-W37,P,2024-09-09,mp
17,5,2024-W24,870,#66301444,2024-06-10,2024-W45,I,2024-11-04,mp


In [33]:
from datetime import date

# TODO: Cambiar a fecha móvil
today = datetime.now()
next_year = today.year
jan_1 = date(next_year, 1, 1)
dec_31 = date(next_year, 12, 31)

dec_31

datetime.date(2024, 12, 31)

In [16]:
available_components = ["bp", "cd", "mt", "st", "cms", "cl", "mp"]
cc_df = read_cc()
cc_df = cc_df.loc[
    ~(
        (cc_df["component_code"] == "mt")
        & ~(cc_df["subcomponente"].str.contains("MOTOR TRACCIÓN"))
    )
]
pool_proj_df = read_base_pool_proj().drop(columns=["changeout_date"])
cc_df = cc_df.loc[cc_df["component_code"].isin(available_components)].reset_index(
    drop=True
)
pool_proj_df = pool_proj_df.loc[
    pool_proj_df["component_code"].isin(available_components)
].reset_index(drop=True)

C:\Users\andmn\DataspellProjects\koapp\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [18]:
df = generate_pool_projection(cc_df, pool_proj_df)
df

Unable to find an available pool slot for changeout on component_code                      cms
changeout_date      2024-06-02 00:00:00
equipo                              863
component_serial                #LNWB01
Name: 3, dtype: object
Unable to find an available pool slot for changeout on component_code                      cms
changeout_date      2024-06-02 00:00:00
equipo                              863
component_serial                   #213
Name: 4, dtype: object
Unable to find an available pool slot for changeout on component_code                      cms
changeout_date      2024-06-08 00:00:00
equipo                              861
component_serial               #PLP0031
Name: 8, dtype: object
Unable to find an available pool slot for changeout on component_code                      cms
changeout_date      2024-06-08 00:00:00
equipo                              861
component_serial                   #106
Name: 9, dtype: object
Unable to find an available pool slot for change

,pool_slot,changeout_week,equipo,component_serial,arrival_week,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
0,1,2023-W27,856,#EG08090003,2023-W40,2023-10-02,bp,Blower,Blower,DERECHO,2023-07-03,5726.0,20000.0,P
1,2,2023-W27,881,#EN12040115,2023-W34,2023-08-21,bp,Blower,Blower,IZQUIERDO,2023-07-05,24180.0,20000.0,P
43,1,2023-W27,870,#EMYK00473,2023-W34,2023-08-21,cd,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,2023-07-05,12829.0,12000.0,P
44,2,2023-W27,851,#EMWG300,2023-W34,2023-08-21,cd,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,2023-07-05,11368.0,12000.0,P
45,3,2023-W27,856,#EMYK00482,2023-W41,2023-10-09,cd,Cilindro_Dirección,Cilindro Dirección,DERECHO,2023-07-07,8237.0,12000.0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,10,2024-W31,397,#EMTB037,2024-W45,2024-11-04,cd,Cilindro_Dirección,Cilindro Dirección,DERECHO,2024-07-31,8241.0,12000.0,P
78,11,2024-W31,864,#EMWJ361,2024-W38,2024-09-16,cd,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,2024-08-01,13592.0,12000.0,P
166,5,2024-W31,850,#201-1,2024-W49,2024-12-02,cms,CMS,Suspension Delantera,DERECHO,2024-08-04,4724.0,15000.0,I
193,16,2024-W31,850,#LNTC11,2024-W41,2024-10-07,cms,CMS,MASA,DERECHO,2024-08-04,4724.0,15000.0,P


In [24]:
pool_proj_df.loc[pool_proj_df["component_code"] == "bp"]

,pool_slot,changeout_week,equipo,component_serial,arrival_week,pool_changeout_type,arrival_date,component_code
0,1,2023-W27,856,#EG08090003,2023-W40,P,2023-10-02,bp
1,2,2023-W27,881,#EN12040115,2023-W34,P,2023-08-21,bp
2,3,2023-W27,875,#EN11110073,2023-W38,P,2023-09-18,bp
3,4,2023-W28,883,#EN12090007,2023-W38,P,2023-09-18,bp
4,6,2023-W29,883,#EE06050200-1,2023-W42,P,2023-10-16,bp
5,5,2023-W29,870,#EE14010117,2023-W42,P,2023-10-16,bp
6,7,2023-W31,399,#EE14091188,2023-W45,P,2023-11-06,bp
7,8,2023-W32,858,#EE14020272,2023-W45,P,2023-11-06,bp
8,9,2023-W33,860,#EE13101143,2023-W47,P,2023-11-20,bp
9,11,2023-W34,882,#EE14080990,2023-W48,P,2023-11-27,bp


In [22]:
df.loc[df["component_code"] == "bp"]

,pool_slot,changeout_week,equipo,component_serial,arrival_week,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
0,1,2023-W27,856,#EG08090003,2023-W40,2023-10-02,bp,Blower,Blower,DERECHO,2023-07-03,5726.0,20000.0,P
1,2,2023-W27,881,#EN12040115,2023-W34,2023-08-21,bp,Blower,Blower,IZQUIERDO,2023-07-05,24180.0,20000.0,P
2,3,2023-W27,875,#EN11110073,2023-W38,2023-09-18,bp,Blower,Blower,IZQUIERDO,2023-07-09,23790.0,20000.0,P
3,4,2023-W28,883,#EN12090007,2023-W38,2023-09-18,bp,Blower,Blower,DERECHO,2023-07-14,17676.0,20000.0,P
5,5,2023-W29,870,#EE14010117,2023-W42,2023-10-16,bp,Blower,Blower,DERECHO,2023-07-17,16263.0,20000.0,P
4,6,2023-W29,883,#EE06050200-1,2023-W42,2023-10-16,bp,Blower,Blower,IZQUIERDO,2023-07-20,8547.0,20000.0,P
6,7,2023-W31,399,#EE14091188,2023-W45,2023-11-06,bp,Blower,Blower,IZQUIERDO,2023-08-06,19176.0,20000.0,P
7,8,2023-W32,858,#EE14020272,2023-W45,2023-11-06,bp,Blower,Blower,DERECHO,2023-08-12,3502.0,20000.0,P
8,9,2023-W33,860,#EE13101143,2023-W47,2023-11-20,bp,Blower,Blower,IZQUIERDO,2023-08-14,12971.0,20000.0,P
10,10,2023-W34,876,#EE13080690,2023-W48,2023-11-27,bp,Blower,Blower,DERECHO,2023-08-21,5652.0,20000.0,P


In [10]:
df.sort_values(["pool_slot", "changeout_date"])

,pool_slot,changeout_week,equipo,component_serial,changeout_date,arrival_week,pool_changeout_type,arrival_date,component_code
0,1,2023-W27,856,#EG08090003,2023-07-03,2023-W40,P,2023-10-02,bp
16,1,2023-W41,868,#EE13111435,2023-10-09,2024-W3,P,2024-01-15,bp
27,1,2024-W6,872,#EE14091124,2024-02-05,2024-W13,P,2024-03-25,bp
38,1,2024-W17,878,#EG07080105,2024-04-22,2024-W24,P,2024-06-10,bp
1,2,2023-W27,881,#EN12040115,2023-07-03,2023-W34,P,2023-08-21,bp
13,2,2023-W38,396,#EE14010094,2023-09-18,2023-W49,P,2023-12-04,bp
22,2,2023-W50,865,#EE06040239,2023-12-11,2024-W12,P,2024-03-18,bp
39,2,2024-W18,864,#EE14070950,2024-04-29,2024-W25,P,2024-06-17,bp
2,3,2023-W27,875,#EN11110073,2023-07-03,2023-W38,P,2023-09-18,bp
14,3,2023-W39,861,#EN12090076,2023-09-25,2024-W1,P,2024-01-01,bp


In [3]:
blob_service_client = BlobServiceClient(
    account_url=os.environ["AZURE_ACCOUNT_URL"],
    credential=os.environ["AZURE_SAS_TOKEN"],
)
blob_client = blob_service_client.get_blob_client(
    container=os.environ["AZURE_CONTAINER_NAME"],
    blob=f"{os.environ['AZURE_PREFIX']}/PLANIFICACION/POOL/2023.01 Pool Componentes MEL Planificadores.xlsx",
)
blob_data = blob_client.download_blob()
blob_data = BytesIO(blob_data.readall())

In [297]:
wb = openpyxl.load_workbook(blob_data, data_only=True)

sheet = wb["PROYECCIÓN"]

start_row = 20  # Starting from row 2
end_row = 33  # Ending at row 10
start_column = "F"  # Starting from column B
end_column = "EF"  # Ending at column E

# Create a DataFrame from the Excel data
sheet = sheet[f"{start_column}{start_row}:{end_column}{end_row}"]
data = []  # .iter_rows(min_row=20, values_only=True)
for row in sheet:
    data.append([cell.value for cell in row])

df = pd.DataFrame(data, columns=[cell.value for cell in sheet[1]])
df.set_index(df.columns[0], inplace=True)
df = df.iloc[1:]
# Generate date range
date_range = pd.date_range(start="2023-07-03", end="2025-12-31", freq="W-MON")

# Create the list of week strings
week_list = [f"{date.isocalendar()[0]}-W{date.isocalendar()[1]}" for date in date_range]

df.columns = week_list[:-1]
# df = df.dropna(how="all", axis=1)

df

,2023-W27,2023-W28,2023-W29,2023-W30,2023-W31,2023-W32,2023-W33,2023-W34,2023-W35,2023-W36,...,2025-W43,2025-W44,2025-W45,2025-W46,2025-W47,2025-W48,2025-W49,2025-W50,2025-W51,2025-W52
N°1,,,,,,,,,,,,,,,,,,,,,
N°1,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°2,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°3,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°4,None,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°5,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°6,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°7,None,None,None,None,1,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°8,None,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°9,None,None,None,None,None,None,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [298]:
from planification.pool.utils import (
    extract_info,
    idx_to_pool_slot,
    get_weeks_and_comments,
    get_end_week,
)


def get_weeks_and_comments(row, sheet):
    weeks = []
    comments = []
    row_idx = int(row.name.strip("N°"))
    col_idx = 1

    for col, value in row.items():
        # print(row_idx)
        if value == 1:
            weeks.append(col)
            cell = sheet[row_idx][col_idx]
            comments.append(cell.comment.text if cell.comment else "")
        col_idx += 1
    return pd.Series({"weeks": weeks, "comments": comments})


def idx_to_pool_slot(df):
    df = (
        df.rename_axis("pool_slot")
        .reset_index()
        .assign(pool_slot=lambda x: x["pool_slot"].str.strip("N°"))
    )
    return df


def extract_info(comment):
    equipo_match = re.search(r"Equipo:\s*(\d+)", comment)
    ns_match = re.search(r"NS:\s*(#+\w*-?\w*)", comment)
    ns = ns_match.group(1) if ns_match else None
    if ns:
        ns = re.sub(r"^#+", "#", ns)
    equipo = equipo_match.group(1) if equipo_match else None

    return equipo, ns


def get_end_week(row, sheet):
    weeks = []
    pool_changeout_types = []
    row_idx = int(row.name.strip("N°"))
    for col_idx in range(0, row.__len__()):
        if col_idx > 2:
            prev_cell = sheet[row_idx][col_idx - 1]
            cell = sheet[row_idx][col_idx]
            if (
                (prev_cell.fill.fgColor.rgb != cell.fill.fgColor.rgb)
                & (prev_cell.value is None)
                & (prev_cell.fill.fgColor.rgb in ["FFEDBFBB", "FFC5E0B4", "FFE88880"])
                & (prev_cell.fill.fgColor.rgb in ["FFC5E0B4", "FFE88880"])
            ) | (
                (prev_cell.fill.start_color.theme != cell.fill.start_color.theme)
                & (prev_cell.value is None)
                & (prev_cell.fill.start_color.theme == 9)
            ):
                # | (
                #     (prev_cell.fill.fgColor.tint != cell.fill.fgColor.tint)
                #     & (prev_cell.fill.fgColor.tint != 0)
                #     & (prev_cell.value is None)
                #     # & (prev_cell.fill.fgColor.rgb in ["FFC5E0B4", "FFE88880"])
                # ):
                weeks.append(list(row.keys())[col_idx - 2])
                if prev_cell.fill.fgColor.rgb in ["FFEDBFBB", '"FFE88880"']:
                    pool_changeout_types.append("I")
                else:
                    pool_changeout_types.append("P")

    return pd.Series({"weeks": weeks, "pool_changeout_type": pool_changeout_types})


# Process the dataframe
changeouts_df = (
    df.apply(lambda x: get_weeks_and_comments(x, filtered_sheet), axis=1)
    .explode(["weeks", "comments"])
    .rename(columns={"weeks": "changeout_week"})
    .pipe(idx_to_pool_slot)
)

# Extract information from comments
changeouts_df[["equipo", "component_serial"]] = changeouts_df["comments"].apply(
    lambda x: pd.Series(extract_info(x))
)

changeouts_df = changeouts_df.drop(columns=["comments"]).assign(
    changeout_date=changeouts_df["changeout_week"].map(
        lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")
    )
)
arrivals_df = (
    df.apply(lambda x: get_end_week(x, filtered_sheet), axis=1)
    .explode(["weeks", "pool_changeout_type"])
    .rename(columns={"weeks": "arrival_week"})
    .pipe(idx_to_pool_slot)
)
arrivals_df = arrivals_df.assign(
    arrival_date=arrivals_df["arrival_week"].map(
        lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")
    )
)

df = pd.merge_asof(
    changeouts_df.sort_values("changeout_date"),
    arrivals_df.sort_values("arrival_date"),
    by="pool_slot",
    left_on="changeout_date",
    right_on="arrival_date",
    direction="forward",
)  # .assign(component_code=file.split("-")[-1].split(".")[0])


changeouts_df

,pool_slot,changeout_week,equipo,component_serial,changeout_date
0,1,2023-W27,856,#EG08090003,2023-07-03
1,1,2023-W41,868,#EE13111435,2023-10-09
2,1,2024-W6,872,#EE14091124,2024-02-05
3,1,2024-W17,878,#EG07080105,2024-04-22
4,2,2023-W27,881,#EN12040115,2023-07-03
5,2,2023-W38,396,#EE14010094,2023-09-18
6,2,2023-W50,865,#EE06040239,2023-12-11
7,2,2024-W18,864,#EE14070950,2024-04-29
8,3,2023-W27,875,#EN11110073,2023-07-03
9,3,2023-W39,861,#EN12090076,2023-09-25


In [299]:
arrivals_df

,pool_slot,arrival_week,pool_changeout_type,arrival_date
0,1,2023-W40,P,2023-10-02
1,1,2024-W3,P,2024-01-15
2,1,2024-W13,P,2024-03-25
3,1,2024-W24,P,2024-06-10
4,2,2023-W34,P,2023-08-21
5,2,2023-W49,P,2023-12-04
6,2,2024-W12,P,2024-03-18
7,2,2024-W25,P,2024-06-17
8,3,2023-W38,P,2023-09-18
9,3,2024-W1,P,2024-01-01


In [289]:
sheet[4][59].fill.start_color.theme

Values must be of type <class 'int'>

In [276]:
sheet[1][57].fill.start_color.theme

Values must be of type <class 'int'>

In [272]:
sheet[1][50].fill.start_color.theme

9

In [224]:
filtered_sheet[1][40].fill.start_color.theme

0

In [200]:
filtered_sheet[1][40].fill.fgColor

<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=0, tint=0.0, type='theme'

In [120]:
filtered_sheet[1].__len__()

131

In [118]:
filtered_sheet[1][100].fill.fgColor.rgb

Values must be of type <class 'str'>

In [105]:
filtered_sheet[1][15].fill.fgColor.rgb

'FFFF0000'

In [59]:
changeouts_df.comments.iloc[0]

'[Threaded comment]\n\nYour version of Excel allows you to read this threaded comment; however, any edits to it will get removed if the file is opened in a newer version of Excel. Learn more: https://go.microsoft.com/fwlink/?linkid=870924\n\nComment:\n    Equipo: 856\nNS: #EG08090003\nEstatus: \nReply:\n    Estatus:  En proceso de armado 12-12-23'

In [35]:
df

,componente,2023-W27,2023-W28,2023-W29,2023-W30,2023-W31,2023-W32,2023-W33,2023-W34,2023-W35,...,2025-W43,2025-W44,2025-W45,2025-W46,2025-W47,2025-W48,2025-W49,2025-W50,2025-W51,2025-W52
0,None,W27,W28,W29,W30,W31,W32,W33,W34,W35,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52
1,N°1,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,N°2,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,N°3,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,N°4,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,N°5,None,None,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,N°6,None,None,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,N°7,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,N°8,None,None,None,None,None,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,N°9,None,None,None,None,None,None,1,None,None,...,None,None,None,None,None,None,None,None,None,None


In [25]:
df.columns

Index([   1, None, None, None, None, None, None, None, None, None,
       ...
       None, None, None, None, None, None, None, None, None, None],
      dtype='object', length=130)

In [23]:
week_list.__len__()

131

In [18]:
df

,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N°1,,,,,,,,,,,,,,,,,,,,,
None,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52
N°1,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°2,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°3,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°4,None,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°5,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°6,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°7,None,None,None,None,1,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°8,None,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [10]:
df

,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N°1,,,,,,,,,,,,,,,,,,,,,
None,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52
N°1,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°2,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°3,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°4,None,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°5,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°6,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°7,None,None,None,None,1,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°8,None,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
